# Import

In [1]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

from ClusterPipe import clustpipe

# Parameters

In [2]:
binned = True
stacked = True

In [3]:
if binned:
    if stacked:
        outdir = os.getenv('CTAPHYS_OUT_DIR')+'KESACCO_binned_stack'
    else:
        outdir = os.getenv('CTAPHYS_OUT_DIR')+'KESACCO_binned_unstack'
else:
    outdir = os.getenv('CTAPHYS_OUT_DIR')+'KESACCO_unbinned'

In [4]:
outdir = outdir+'_eminmax'

### Define the simulation

In [5]:
cpipe = clustpipe.ClusterPipe(silent=False, output_dir=outdir)

             _  __  ___    __    __     ___   ___   __         
            | |/ / | __| /' _/  /  \   / _/  / _/  /__\        
            |   <  | _|  `._`. | /\ | | \__ | \__ | \/ |       
            |_|\_\ |___| |___/ |_||_|  \__/  \__/  \__/        
Keen Event Simulation and Analysis for CTA Cluster Observations
---------------------------------------------------------------


### Define the cluster object

In [6]:
cpipe.cluster.name = 'Coma'
cpipe.cluster.redshift = 0.0179
cpipe.cluster.M500 = 6.2e14*u.solMass
cpipe.cluster.coord = SkyCoord("194.953d  27.9806d", frame="icrs", unit="deg")

# CR physics
cpipe.cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}
cpipe.cluster.set_density_crp_isodens_scal_param(scal=1.0)
cpipe.cluster.X_crp_E = {'X':1.0, 'R_norm':cpipe.cluster.R500}

# Sampling
cpipe.cluster.Npt_per_decade_integ = 30

# Get information
cpipe.cluster.print_param()

--- theta_truncation
    0.6 deg
    <class 'astropy.units.quantity.Quantity'>
--- abundance
    0.3
    <type 'float'>
--- R_truncation
    0.811596518453 Mpc
    <class 'astropy.units.quantity.Quantity'>
--- M500
    8.39114776012e+13 solMass
    <class 'astropy.units.quantity.Quantity'>
--- map_reso
    0.01 deg
    <class 'astropy.units.quantity.Quantity'>
--- theta500
    0.5 deg
    <class 'astropy.units.quantity.Quantity'>
--- Rmin
    1.0 kpc
    <class 'astropy.units.quantity.Quantity'>
--- magfield_model
    {'a': 1.33, 'c500': 1.81, 'c': 0.155, 'b': 2.065, 'name': 'GNFW', 'r_p': <Quantity 855.62300911 kpc>, 'P_0': <Quantity 10. uG>}
    <type 'dict'>
--- density_gas_model
    {'name': 'doublebeta', 'beta2': 0.87, 'beta1': 1.8, 'n_01': <Quantity 0.039 1 / cm3>, 'n_02': <Quantity 0.00405 1 / cm3>, 'r_c1': <Quantity 80. kpc>, 'r_c2': <Quantity 280. kpc>}
    <type 'dict'>
--- Epmin
    1.21793391659 GeV
    <class 'astropy.units.quantity.Quantity'>
--- pp_interaction_model
    

### Define the point source object

In [7]:
# source 1
name='NGC1275'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h19m48.16s +41d30m42s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h19m48.16s +41d30m42s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':2.1e-11/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-3.6, 'free':True},
                   'PivotEnergy':{'value':0.2*u.TeV, 'free':False}}}
cpipe.compact_source.add_source(name, spatial, spectral)

In [8]:
# source 2
name='IC310'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h16m42.98s +41d19m30s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h16m42.98s +41d19m30s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':4.3e-12/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-1.95, 'free':True},
                   'PivotEnergy':{'value':1.0*u.TeV, 'free':False}}}
cpipe.compact_source.add_source(name, spatial, spectral)

In [9]:
cpipe.compact_source.print_source()

--- NGC1275
    -- Spatial model: PointSource
         DEC: {'free': False, 'value': <Latitude 41.51166667 deg>}
         RA: {'free': False, 'value': <Longitude 49.95066667 deg>}
    -- Spectral model: PowerLaw
         Index: {'free': True, 'value': -3.6}
         PivotEnergy: {'free': False, 'value': <Quantity 0.2 TeV>}
         Prefactor: {'free': True, 'value': <Quantity 2.1e-11 1 / (cm2 s TeV)>}
    -- Temporal model: Constant
         Normalization: {'free': False, 'value': 1.0}
--- IC310
    -- Spatial model: PointSource
         DEC: {'free': False, 'value': <Latitude 41.325 deg>}
         RA: {'free': False, 'value': <Longitude 49.17908333 deg>}
    -- Spectral model: PowerLaw
         Index: {'free': True, 'value': -1.95}
         PivotEnergy: {'free': False, 'value': <Quantity 1. TeV>}
         Prefactor: {'free': True, 'value': <Quantity 4.3e-12 1 / (cm2 s TeV)>}
    -- Temporal model: Constant
         Normalization: {'free': False, 'value': 1.0}


### Define the observations

In [10]:
# One pointing centered on cluster
cpipe.obs_setup.add_obs(obsid='011', name='Perseus_Ptg1', 
                        coord=cpipe.cluster.coord, 
                        rad=1*u.deg, 
                        emin=0.05*u.TeV, emax=10*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T00:00:00.0', tmax='2020-01-01T01:00:00.0', deadc=0.95)

# One pointing offset +0+1
cpipe.obs_setup.add_obs(obsid='012', name='Perseus_Ptg2', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value+1, unit='deg'),
                        rad=1*u.deg,
                        emin=0.05*u.TeV, emax=10*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-02T00:00:00.0', tmax='2020-01-02T01:00:00.0', deadc=0.95)

# One pointing offset +0-1
cpipe.obs_setup.add_obs(obsid='013', name='Perseus_Ptg3', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value-1, unit='deg'),
                        rad=1*u.deg, 
                        emin=0.05*u.TeV, emax=10*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-03T00:00:00.0', tmax='2020-01-03T01:00:00.0', deadc=0.95)

# One pointing offset +1+0
cpipe.obs_setup.add_obs(obsid='014', name='Perseus_Ptg4', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+1, cpipe.cluster.coord.dec.value+0, unit='deg'),
                        rad=1*u.deg,
                        emin=0.05*u.TeV, emax=10*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-04T00:00:00.0', tmax='2020-01-04T01:00:00.0', deadc=0.95)

# One pointing offset -1-0
cpipe.obs_setup.add_obs(obsid='015', name='Perseus_Ptg5', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value-1, cpipe.cluster.coord.dec.value+0, unit='deg'),
                        rad=1*u.deg, 
                        emin=0.05*u.TeV, emax=10*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-05T00:00:00.0', tmax='2020-01-05T01:00:00.0', deadc=0.95)

# Print info
cpipe.obs_setup.print_obs()

=== Perseus_Ptg1, ObsID 011
        RA-Dec:    49.9466666667, 41.5130555556 deg
        GLON-GLAT: 150.572480744, -13.2617332972 deg
        ROI rad: 1.0 deg
        tmin: 2020-01-01T00:00:00.0
        tmax: 2020-01-01T01:00:00.0
        emin: 0.05 TeV
        emax: 10.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg2, ObsID 012
        RA-Dec:    49.9466666667, 42.5130555556 deg
        GLON-GLAT: 150.007878416, -12.4268776697 deg
        ROI rad: 1.0 deg
        tmin: 2020-01-02T00:00:00.0
        tmax: 2020-01-02T01:00:00.0
        emin: 0.05 TeV
        emax: 10.0 TeV
        deadc: 0.95
        caldb: prod3b-v2
        irf: North_z20_S_5h
        bkg: name Background, obsid None, spatial type CTAIrfBackground, spectral type PowerLaw
=== Perseus_Ptg3, ObsID 013
        RA-Dec:    49.9466666667, 40.5130555556 deg
        GLON-GLAT: 151.1409715

# Run the simulation

In [11]:
cpipe.cluster.map_reso = 0.1*u.deg

cpipe.run_sim_obs()
cpipe.run_sim_quicklook(ShowEvent=True, ShowObsDef=True, ShowSkyModel=False,
                        bkgsubtract='IRF', smoothing_FWHM=0.2*u.deg)

----- ObsID to be observed: ['011', '012', '013', '014', '015']
         than 0.01 deg, while the PSF at 100 TeV is ~0.02 deg.    

------- Simulation log -------
=== GApplication ===
 Name ......................: ctobssim
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Sim_ObsDef.xml
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Sim_Model_Unstack.xml
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 edisp .....................: no
 outevents .................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Events.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/TmpEvents
 startindex ................: 1
 seed ......................: 944140
 ra ........................: 50.9466666666667
 dec .......................: 42.5130555555556
 rad .............

----- Quicklook 3/5 done
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Events014.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Sim_Skymap014.fits
 emin ......................: 0.05
 emax ......................: 10
 usepnt ....................: no
 nxpix .....................: 11
 nypix .....................: 11
 binsz .....................: 0.1
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: IRF
 roiradius .................: 0.1
 inradius ..................: 1
 outradius .................: 2
 iterations ................: 3
 thres

# Run the analysis

In [12]:
#----- Analysis parameters
cpipe.method_binned = binned
cpipe.method_stack  = stacked
cpipe.spec_enumbins = 10

cpipe.map_reso          = 0.1*u.deg
cpipe.map_fov           = 1.5*u.deg
cpipe.map_coord         = copy.deepcopy(cpipe.cluster.coord)
cpipe.cluster.map_reso  = copy.deepcopy(cpipe.map_reso)
cpipe.cluster.map_fov   = 2*cpipe.cluster.theta_truncation
cpipe.cluster.map_coord = copy.deepcopy(cpipe.cluster.coord)
cpipe.map_UsePtgRef     = False

obsID = ['011', '012', '013', '014', '015']

In [13]:
cpipe.run_analysis(obsID=obsID,
                   refit=False, like_accuracy=0.05, max_iter=50, fix_spat_for_ts=False,
                   imaging_bkgsubtract='NONE', do_Skymap=True, do_SourceDet=True, do_ResMap=True, do_TSmap=False, profile_reso=0.05*u.deg,
                   do_Spec=True, do_Butterfly=True, do_Resspec=True,
                   smoothing_FWHM=0.1*u.deg, profile_log=True)

----- ObsID to be analysed: ['011', '012', '013', '014', '015']

=== GApplication ===
 Name ......................: ctselect
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Events.xml
 outobs ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_EventsSelected.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Selected
 usepnt ....................: no
 ra ........................: 49.9466666666667
 dec .......................: 41.5130555555556
 rad .......................: 1.06066017177982
 forcesel ..................: yes
 tmin ......................: NONE
 tmax ......................: NONE
 emin ......................: 0.05
 emax ......................: 100
 phase .....................: NONE
 expr ......................: 
 usethres ..................: NONE
 publish ...................: no
 chatter ..........

 logfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Model_Cube_log.txt

=== GApplication ===
 Name ......................: ctmodel
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Countscube.fits
 incube ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Model_Output_Cluster.xml
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Bkgcube.fits
 caldb .....................: NONE
 irf ..............

=== GApplication ===
 Name ......................: csspec
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Model_Output.xml
 srcname ...................: Perseus
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Bkgcube.fits
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Spectrum_Perseus.fits
 method ....................:

 logfile ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Spectrum_Residual_log.txt

=== GApplication ===
 Name ......................: ctmodel
 Version ...................: 1.6.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Countscube.fits
 incube ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Countscube.fits
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Expected_Cluster.xml
 expcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Expcube.fits
 psfcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Psfcube.fits
 edispcube .................: NONE
 bkgcube ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_binned_stack_eminmax/Ana_Bkgcube.fits
 caldb .....................: NONE
 irf ...........

         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.
         This assumes weak noise spatial variarion (w.r.t. smoothing),
         gaussian regime, and uncorrelated pixels.


/Users/adam/Project/CTA/Phys/Software/ClusterPipe/Tools/plotting.py:523: RuntimeWarning: invalid value encountered in greater_equal
  w_pos = prof['profile'] >= 0
/Users/adam/Project/CTA/Phys/Software/ClusterPipe/Tools/plotting.py:524: RuntimeWarning: invalid value encountered in less
  w_neg = prof['profile'] < 0
/Users/adam/Project/anaconda3/envs/cta/lib/python2.7/site-packages/matplotlib/axes/_base.py:3477: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


In [14]:
#cpipe.run_ana_dataprep(obsID=obsID)

In [15]:
#cpipe.run_ana_likelihood(refit=False, like_accuracy=0.05, max_iter=50, fix_spat_for_ts=False)

In [16]:
#cpipe.run_ana_imaging(bkgsubtract='NONE', do_TS=False, do_Res=True, do_Skymap=True, do_SourceDet=False)

In [17]:
#cpipe.run_ana_spectral(do_Spec=True, do_Butterfly=True, do_Res=True)

In [18]:
#cpipe.run_ana_expected_output(obsID=obsID, profile_reso=0.05*u.deg)

In [19]:
#cpipe.run_ana_plot(obsID=obsID, profile_log=True, smoothing_FWHM=0.1*u.deg)